In [ ]:
!pip install -q pyomo
!apt-get install -y -qq glpk-utils

In [1]:
import pyomo.environ as pyo

# Create model
model = pyo.ConcreteModel()

# Sets
model.I = pyo.Set(initialize=[1, 2, 3])  # i set
model.T = pyo.Set(initialize=[1, 2, 3, 4])  # t set

# Parameters
p_i_t = {(1, 1): 15000, (1, 2): 15000, (1, 3): 15000, (1, 4): 15000,
         (2, 1): 12000, (2, 2): 12000, (2, 3): 12000, (2, 4): 12000,
         (3, 1): 10000, (3, 2): 10000, (3, 3): 10000, (3, 4): 10000}


k_i_t = {(1, 1): 200, (1, 2): 50, (1, 3): 300, (1, 4): 100,
         (2, 1): 100, (2, 2): 500, (2, 3): 200, (2, 4): 400,
         (3, 1): 300, (3, 2): 400, (3, 3): 100, (3, 4): 500} # shows the demand of bread i in day t

# Decision variables
model.x = pyo.Var(model.I, domain=pyo.NonNegativeReals)
model.y = pyo.Var(model.I, model.T, domain=pyo.NonNegativeIntegers)
model.yprime = pyo.Var(model.I, model.T, domain=pyo.NonNegativeIntegers)
model.f = pyo.Var(model.I, model.T, domain=pyo.NonNegativeIntegers)

# Objective function
def objective_function(model):
    return sum(p_i_t[i, t] * model.f[i, t] for i in model.I for t in model.T) - (20000*model.x[1] + 15000*model.x[2] + 10000*model.x[3])
model.obj = pyo.Objective(rule=objective_function, sense=pyo.maximize)

# Constraints
def sum_x_constraint(model):
    return sum(model.x[i] for i in model.I) <= 800
model.sum_x_constraint = pyo.Constraint(rule=sum_x_constraint)

def x_constraint_1(model):
    return (0.7 * 0.25 * sum(model.y[1, t] for t in model.T) +
            0.3 * 0.25 * sum(model.y[2, t] for t in model.T) +
            0.1 * 0.25 * sum(model.y[3, t] for t in model.T)) <= model.x[1]
model.x_constraint_1 = pyo.Constraint(rule=x_constraint_1)

def x_constraint_2(model):
    return (0.2 * 0.25 * sum(model.y[1, t] for t in model.T) +
            0.5 * 0.25 * sum(model.y[2, t] for t in model.T) +
            0.3 * 0.25 * sum(model.y[3, t] for t in model.T)) <= model.x[2]
model.x_constraint_2 = pyo.Constraint(rule=x_constraint_2)

def x_constraint_3(model):
    return (0.1 * 0.25 * sum(model.y[1, t] for t in model.T) +
            0.2 * 0.25 * sum(model.y[2, t] for t in model.T) +
            0.6 * 0.25 * sum(model.y[3, t] for t in model.T)) <= model.x[3]
model.x_constraint_3 = pyo.Constraint(rule=x_constraint_3)


def sum_y_constraints(model, t):
    return sum(model.y[i, t] for i in model.I) <= 800
model.sum_y_constraints = pyo.Constraint(model.T, rule=sum_y_constraints)

def f_constraint(model, i, t):
    return model.f[i, t] <= k_i_t[i, t]
model.f_constraints = pyo.Constraint(model.I, model.T, rule=f_constraint)

def y_f_relationship_1(model):
    return model.y[1, 1] == model.yprime[1, 1] + model.f[1, 1]
model.y_f_relationship_1 = pyo.Constraint(rule=y_f_relationship_1)

def y_f_relationship_2(model):
    return model.y[2, 1] == model.yprime[2, 1] + model.f[2, 1]
model.y_f_relationship_2 = pyo.Constraint(rule=y_f_relationship_2)

def y_f_relationship_3(model):
    return model.y[3, 1] == model.yprime[3, 1] + model.f[3, 1]
model.y_f_relationship_3 = pyo.Constraint(rule=y_f_relationship_3)

def y_f_relationship_4(model):
    return model.y[1, 2] + 0 == model.yprime[1, 2] + model.f[1, 2]
model.y_f_relationship_4 = pyo.Constraint(rule=y_f_relationship_4)

def y_f_relationship_5(model):
    return model.y[2, 2] + model.yprime[1, 1] == model.yprime[2, 2] + model.f[2, 2]
model.y_f_relationship_5 = pyo.Constraint(rule=y_f_relationship_5)

def y_f_relationship_6(model):
    return model.y[3, 2] + model.yprime[2, 1] == model.yprime[3, 2] + model.f[3, 2]
model.y_f_relationship_6 = pyo.Constraint(rule=y_f_relationship_6)

def y_f_relationship_7(model):
    return model.y[1, 3] + 0 == model.yprime[1, 3] + model.f[1, 3]
model.y_f_relationship_7 = pyo.Constraint(rule=y_f_relationship_7)

def y_f_relationship_8(model):
    return model.y[2, 3] + model.yprime[1, 2] == model.yprime[2, 3] + model.f[2, 3]
model.y_f_relationship_8 = pyo.Constraint(rule=y_f_relationship_8)

def y_f_relationship_9(model):
    return model.y[3, 3] + model.yprime[2, 2] == model.yprime[3, 3] + model.f[3, 3]
model.y_f_relationship_9 = pyo.Constraint(rule=y_f_relationship_9)

def y_f_relationship_10(model):
    return model.y[1, 4] + 0 == model.yprime[1, 4] + model.f[1, 4]
model.y_f_relationship_10 = pyo.Constraint(rule=y_f_relationship_10)

def y_f_relationship_11(model):
    return model.y[2, 4] + model.yprime[1, 3] == model.yprime[2, 4] + model.f[2, 4]
model.y_f_relationship_11 = pyo.Constraint(rule=y_f_relationship_11)

def y_f_relationship_12(model):
    return model.y[3, 4] + model.yprime[2, 3] == model.yprime[3, 4] + model.f[3, 4]
model.y_f_relationship_12 = pyo.Constraint(rule=y_f_relationship_12)


model.yprime_3_1_zero = pyo.Constraint(expr=model.yprime[3, 1] == 0)
model.yprime_3_2_zero = pyo.Constraint(expr=model.yprime[3, 2] == 0)
model.yprime_3_3_zero = pyo.Constraint(expr=model.yprime[3, 3] == 0)
model.yprime_3_4_zero = pyo.Constraint(expr=model.yprime[3, 4] == 0)


# Solve the model
solver = pyo.SolverFactory('glpk')
result = solver.solve(model, tee=False)

print(result)

# Display the results
model.display()

# for i in model.I:
#     for t in model.T:
#         print(f"x_{i} = {model.x[i].value}")
#         print(f"y_{i},{t} = {model.y[i, t].value}")
#         print(f"yprime_{i},{t} = {model.yprime[i, t].value}")
#         print(f"f_{i},{t} = {model.f[i, t].value}")



Problem: 
- Name: unknown
  Lower bound: 25293750.0
  Upper bound: 25293750.0
  Number of objectives: 1
  Number of constraints: 36
  Number of variables: 39
  Number of nonzeros: 112
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.0043146610260009766
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Model unknown

  Variables:
    x : Size=3, Index=I
        Key : Lower : Value  : Upper : Fixed : Stale : Domain
          1 :     0 : 271.25 :  None : False : False : NonNegativeReals
          2 :     0 : 253.75 :  None : False : False : NonNegativeReals
          3 :     0 :  262.5 :  None : False : False : NonNegativeReals
    y : Size=12, Index=I*T
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 1) :     0 : 350.0 :  None : False : False : NonNegativeIntegers
        (1, 